# python 获取股票数据可视化

## tushare 

### 个股行情数据：

In [6]:
#先引入后面分析、可视化等可能用到的库
import tushare as ts
import pandas as pd  
import matplotlib.pyplot as plt
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

import json
from pathlib import Path

"""
{
    "tushare_token": ""
}
"""
source_path = Path("./py_assets_data_config.json")
configs = json.loads(source_path.read_text())
#设置token

# token='你的token'
token = configs['tushare_token']
#ts.set_token(token)
pro = ts.pro_api(token)

In [18]:
#获取当前上市的股票代码、简称、注册地、行业、上市时间等数据
basic=pro.stock_basic(list_status='L')
#查看前五行数据
#basic.head(5)

#获取平安银行日行情数据
pa=pro.daily(ts_code='000001.SZ', start_date='20180101',
               end_date='20190106')
#pa.head()

#K线图可视化
from pyecharts import options as opts
from pyecharts.charts import Kline
pa.index=pd.to_datetime(pa.trade_date)
pa=pa.sort_index()
v1=list(pa.loc[:,['open','close','low','high']].values)
t=pa.index
v0=list(t.strftime('%Y%m%d'))
kline = Kline(
# "平安银行K线图"
# , title_text_size=15
)
# kline.add("", v0, v1,is_datazoom_show=True,
#          mark_line=["average"],
#          mark_point=["max", "min"],
#          mark_point_symbolsize=60,
#          mark_line_valuedim=['highest', 'lowest'] )
kline.add_xaxis(v0)
kline.add_yaxis(
    "Kline",
    v1,
    markpoint_opts=opts.MarkPointOpts(data=[ opts.MarkPointItem(value_dim=["max", "min"]) ]),
    # markline_opts=opts.MarkLineOpts(type_='max', value_dim="close")
    markline_opts=opts.MarkLineOpts(data=[ opts.MarkLineItem(name="average", value_dim=['highest',"lowest"]) ])
)

kline.set_global_opts(
    xaxis_opts=opts.AxisOpts(is_scale=True),
    yaxis_opts=opts.AxisOpts(
        is_scale=True,
        splitarea_opts=opts.SplitAreaOpts(is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1))
    ),
    title_opts=opts.TitleOpts(title= "平安银行K线图"),
)
#kline.render("上证指数图.html")
kline.render()
# kline.render_notebook()

'/Users/afirez/studio/python/auto-trade/examples/py_quant/render.html'

In [19]:
#定义获取多只股票函数：
def get_stocks_data(stocklist,start,end):
    all_data={}
    for code in stocklist:
        all_data[code]=pro.daily(ts_code=code,
                 start_date=start, end_date=end)
    return all_data

#保存本地
def save_data(all_data):
    for code,data in all_data.items():
        data.to_csv('./data/'+code+'.csv',
                     header=True, index=False)

stocklist=list(basic.ts_code)[:15]
start=''
end=''
all_data=get_stocks_data(stocklist,start,end)

all_data['000002.SZ'].tail()

#将数据保存到本地
save_data(all_data)

#读取本地文件夹里所有文件
import os
#文件存储路径
file='./data/'
g=os.walk(file)
filenames=[]
for path,d,filelist in g:
    for filename in filelist:
        filenames.append(os.path.join(filename))
print(filenames)

#将读取的数据文件放入一个字典中
df={}
#从文件名中分离出股票代码
code=[name.split('.')[0] for name in filenames]
for i in range(len(filenames)):
    filename=file+filenames[i]
    df[code[i]]=pd.read_csv(filename)

#查看第一只股票前五行数据
#df[code[0]].head()

['000010.SZ.csv', '000002.SZ.csv', '000012.SZ.csv', '600288SH.csv', '000016.SZ.csv', '000008.SZ.csv', '000004.SZ.csv', '000006.SZ.csv', '000014.SZ.csv', '000011.SZ.csv', '000001.SZ.csv', '600288SH_ts.csv', '000009.SZ.csv', '000017.SZ.csv', '000005.SZ.csv', '000007.SZ.csv', '000019.SZ.csv']


In [20]:
import numpy as np

def get_index_data(indexs):
    '''indexs是字典格式'''
    index_data={}
    for name,code in indexs.items():
        df=pro.index_daily(ts_code=code)
        df.index=pd.to_datetime(df.trade_date)   
        index_data[name]=df.sort_index()
    return index_data

#获取常见股票指数行情
indexs={'上证综指': '000001.SH','深证成指': '399001.SZ',
         '沪深300': '000300.SH','创业板指': '399006.SZ',
          '上证50': '000016.SH', '中证500': '000905.SH',
         '中小板指': '399005.SZ','上证180': '000010.SH'}
index_data=get_index_data(indexs)
#index_data['上证综指'].head()

#对股价走势进行可视化分析
subjects =list(index_data.keys())
#每个子图的title
plot_pos = [421,422,423,424,425,426,427,428] # 每个子图的位置
new_colors = ['#1f77b4','#ff7f0e', '#2ca02c', '#d62728',
             '#9467bd','#8c564b', '#e377c2', 
             '#7f7f7f','#bcbd22','#17becf']

fig = plt.figure(figsize=(16,18))
fig.suptitle('A股股指走势',fontsize=18)
for pos in np.arange(len(plot_pos)):       
    ax = fig.add_subplot(plot_pos[pos]) 
    y_data =index_data[subjects[pos]]['close']    
    b = ax.plot(y_data,color=new_colors[pos])   
    ax.set_title(subjects[pos])    
    # 将右上边的两条边颜色设置为空，相当于抹掉这两条边
    ax = plt.gca()  
    ax.spines['right'].set_color('none') 
    ax.spines['top'].set_color('none')    
plt.show()

Exception: 抱歉，您没有访问该接口的权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。